In [1]:
#import libraries
from pymongo import MongoClient
import pandas as pd

In [11]:


def geoquery(lat, long, radius):
    client = MongoClient ('localhost', 27017)
    db = client['companies_clean']

    loc = {"lng": long, "lat": lat}

    result = pd.DataFrame(db.comps.find({
        "coord": {
         "$near": {
           "$geometry": {
              "type": "Point" ,
              "coordinates": [ loc["lng"] , loc["lat"] ]
           },
           "$maxDistance": radius, # In meters 
         }
        }}))
    return result



In [16]:
'''
london = geoquery(51.507410, -0.127685, 5000)
san_francisco = geoquery(37.779248, -122.419243, 5000)
new_york = geoquery(40.725900, -73.981510, 5000)
denver = geoquery(39.739266, -104.987428, 5000)
madrid = geoquery(40.417002, -3.703521, 5000)
locations_list = [london, san_francisco, new_york, denver, madrid]
'''

,_id,category_code,coord,founded_year,latitude,longitude,name,number_of_employees,raised_amount,raised_currency_code,total_money_raised
0,5cd5942a666d28a915fcda50,software,"[-0.1262362, 51.5001524]",2008,51.500152,-0.126236,TweetDeck,5.0,300000.0,USD,3800000.0
1,5cd5942a666d28a915fcdc6a,enterprise,"[-0.1262362, 51.5001524]",2008,51.500152,-0.126236,oneDrum,5.0,1000000.0,GBP,1000000.0
2,5cd5942a666d28a915fcd911,finance,"[-0.1418973, 51.5108803]",2007,51.510880,-0.141897,Seedcamp,4.0,3000000.0,EUR,5000000.0
3,5cd5942a666d28a915fcdc6d,advertising,"[-0.1366861, 51.5152023]",2008,51.515202,-0.136686,Struq,50.0,8500000.0,USD,8500000.0
4,5cd5942a666d28a915fcdc0f,advertising,"[-0.1067217, 51.5080607]",2006,51.508061,-0.106722,Specle,9.0,149991.0,GBP,897000.0


,_id,category_code,coord,founded_year,latitude,longitude,name,number_of_employees,raised_amount,raised_currency_code,total_money_raised
0,5cd5942a666d28a915fcd8d8,games_video,"[-122.409173, 37.788482]",2006,37.788482,-122.409173,Kyte,40.0,2250000.0,USD,2.340000e+07
1,5cd5942a666d28a915fcd8d9,advertising,"[-118.1327468, 34.1494713]",2008,34.149471,-118.132747,OpenX,305.0,5000000.0,USD,7.550000e+07
2,5cd5942a666d28a915fcd8da,games_video,"[-74.0094471, 40.7464969]",2006,40.746497,-74.009447,Joost,0.0,45000000.0,USD,4.500000e+07
3,5cd5942a666d28a915fcd8db,games_video,"[-73.995625, 40.726155]",2007,40.726155,-73.995625,Livestream,120.0,1200000.0,USD,1.470000e+07
4,5cd5942a666d28a915fcd8dc,games_video,"[-74.3372, 40.801358]",2007,40.801358,-74.337200,AdaptiveBlue,15.0,1500000.0,USD,2.400000e+07
5,5cd5942a666d28a915fcd8dd,web,"[-118.487267, 34.017606]",2007,34.017606,-118.487267,Mahalo,40.0,5000000.0,USD,2.100000e+07
6,5cd5942a666d28a915fcd8de,mobile,"[-121.932202, 37.465645]",2006,37.465645,-121.932202,GrandCentral,3.0,4000000.0,USD,4.000000e+06
7,5cd5942a666d28a915fcd8df,search,"[-122.395289, 37.778613]",2006,37.778613,-122.395289,Powerset,60.0,12500000.0,USD,2.250000e+07
8,5cd5942a666d28a915fcd8e0,games_video,"[-122.07948, 37.392936]",2007,37.392936,-122.079480,Ustream,250.0,1700000.0,USD,6.010000e+07
9,5cd5942a666d28a915fcd8e1,search,"[-122.217964, 37.488336]",2006,37.488336,-122.217964,Spock,30.0,7000000.0,USD,7.000000e+06
